In [23]:
import glob
import pickle
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *
from imblearn.over_sampling import SMOTE
from library.preprocessing import new_Processing_before_journal

In [24]:
all_folders=glob.glob("../1. DataProcessing/Trails/*/*/")

In [25]:
test_dates=['2019-06-28','2019-06-30','2019-07-01','2019-07-02','2019-07-03','2019-07-04','2019-07-05']
formatted_test_dates=['-'.join(d.split("-")[::-1]) for d in test_dates] #reversing order

In [26]:
#train folders
train_folders=[f for f in all_folders if f.split("\\")[-2].split("_")[0] not in formatted_test_dates]

#test folders
test_folders=[f for f in all_folders if f.split("\\")[-2].split("_")[0] in formatted_test_dates]

# Modelling

In [27]:
#trqining_data
l=[]
for e in train_folders:
    try:
        data=new_Processing_before_journal(e+"ALL_DATA.csv")
        l.append(data)
    except:
        pass
    
train_df=pd.concat(l)#Raw data is processed

#testing_data
test_df=pd.concat([new_Processing_before_journal(e+"ALL_DATA.csv") for e in test_folders])#Raw data is processed


In [28]:
#feature # binding
F=\
    {1:'stay_duration', #f1
     2:'mfcc0',3:'mfcc1',4:'mfcc2',5:'mfcc3',6:'mfcc4', #f2,f3,f4,f5,f6
     7:'wifi_count', 8:'edge_wifi_count', #f7,f8
     9:'RSI', #f9
     10:'human_made', 11:'natural_land',12:'road_exist_percent',13:'highly_populated_poi_exist'#f10,f11,f12,f13
    }

selected_feat=\
    {
        'Is_Bus_stop':[F[e] for e in [10,12,11,1,8]],
        'Is_Turn':[F[e] for e in [10,12,11,1,8,9]],
        'Is_Signal':[F[e] for e in [12,11,10,8,9]],
        'Is_Congestion':[F[e] for e in [2,11,10,9,12,8,6,1]],
        'Is_Adhoc':[F[e] for e in [10,11,1,12,2]]
    }

In [29]:
#Modeling function
def get_metrics_from_the_model(train,labels_train,test,labels_test,poi):
    rf=RandomForestClassifier(n_estimators=100,max_depth=8,random_state=42)
    rf.fit(train, labels_train)

    pred_train= rf.predict(train)
    pred_test= rf.predict(test)
    
    performance={'poi':poi,
                 'tr_acc':accuracy_score(labels_train,pred_train),
                 'tr_precision':precision_score(labels_train,pred_train,average='weighted'),
                 'tr_recall':recall_score(labels_train,pred_train,average='weighted'),
                 'tr_f1-score':f1_score(labels_train,pred_train,average='weighted'),

                 'te_acc':accuracy_score(labels_test,pred_test),
                 'te_precision':precision_score(labels_test, pred_test,average='weighted'),
                 'te_recall':recall_score(labels_test, pred_test,average='weighted'),
                 'te_f1-score':f1_score(labels_test, pred_test,average='weighted')}
    return performance,rf

In [30]:
perf=[] #performance list
rfs={} #rf dict


for poi_column in ['Is_Bus_stop','Is_Turn','Is_Signal','Is_Congestion','Is_Adhoc']:
    feature_names=selected_feat[poi_column] #feature selection



    #SMOTE on training data & get features ,labels
    X = train_df[feature_names].copy()
    y = train_df[poi_column].copy()

    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)
    train_data = X_resampled.copy()
    train_labels = y_resampled.copy()

    #Test set get features & labels
    test_data = test_df[feature_names].values
    test_labels = test_df[poi_column].values

    #Training with Random Forest
    performance,rf=get_metrics_from_the_model(train_data,train_labels,test_data,test_labels,poi_column)

    #adding to performance list
    perf.append(performance)
    rfs[poi_column]=rf

    print(f'Completed for POI:{poi_column}')

Completed for POI:Is_Bus_stop
Completed for POI:Is_Turn
Completed for POI:Is_Signal
Completed for POI:Is_Congestion
Completed for POI:Is_Adhoc


In [32]:
df=pd.DataFrame(perf)
df.to_csv("./logs/model_performace.csv",index=False)
df

,poi,tr_acc,tr_precision,tr_recall,tr_f1-score,te_acc,te_precision,te_recall,te_f1-score
0,Is_Bus_stop,0.908000,0.908840,0.908000,0.907953,0.834680,0.845378,0.834680,0.836458
1,Is_Turn,0.903067,0.905589,0.903067,0.902916,0.826598,0.854060,0.826598,0.833325
2,Is_Signal,0.968940,0.969707,0.968940,0.968927,0.939015,0.959669,0.939015,0.946014
3,Is_Congestion,0.860352,0.865007,0.860352,0.859905,0.767083,0.949615,0.767083,0.847308
4,Is_Adhoc,0.825852,0.828018,0.825852,0.825564,0.722263,0.721626,0.722263,0.721867


In [33]:
#Saving Models
with open('./logs/rfs.pickle', 'wb') as handle:
    pickle.dump(rfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [34]:
#NICE